In [2]:
import numpy as np
%matplotlib inline

前章のword2vecは語彙数が増えると計算量があまりにもおおくなる  
高速化のため，以下の2つの高速化を行う  
- Embeddingレイヤの導入
- Negative Samplingという損失関数の導入

PTBデータセットという実用的なサイズのコーパスを対象に学習を行う．  
単語の分散表現の良さを実際に評価する．

# まとめ
word2vecは語彙が増えるごとに計算量が増加するので，次の改良を行った
- MatMulレイヤ -> Embeddingレイヤ
    - one-hotベクトルを全部MatMulする必要はない．重みの対応する行を抜き出すだけ
- 出力層を 多値分類 -> 2値分類 + Negative sampling
    - 正解の単語に対する確率と，少量の不正解の単語に対する確率だけ計算すればいい
    - 不正解の単語をサンプリングするにはコーパス中の出現数の分布に0.75などの累乗を施した確率分布からランダムで選び出す

これらは，word2vecの計算の中で必要なところのみを取り扱うことで近似的に答えを導く方法である．  

word2vecによって得られた単語の分散表現(入力層の重み)は
- 似たコンテキストで使われる単語は近い値を持つようになる
- 類推問題をベクトルの加算と減算によって解ける性質を持つ
- 転移学習に利用でき，その他の様々な自然言語処理のタスクに利用できる．

# word2vecの改良①
CBOWモデルでは，例えば語彙が100万のとき，以下の計算がボトルネックとなる．  
- 入力層の100万次元のone-hot表現と重み行列Win(1000000, 100)の積による計算
    - 4.1節のEmbeddingレイヤで解決
- 中間層と重み行列Wout(100, 1000000)の積と100万次元の出力を持つSoftmaxレイヤの計算
    - 4.2節のNegative Samplingで解決

## Embeddingレイヤ
CBOWの入力層とWinの計算は，単にone-hotベクトルで1になっているWinの行を抜き出しているだけ．  
重みパラメータから，単語IDに該当する行ベクトルを抜き出すためのレイヤとして，Embeddingレイヤを作成する．  
word embeddingとは，単語の密なベクトル表現のことで，「単語の埋め込み」や「単語の分散表現(distributed representation)」と呼ばれる．  

## Embeddingレイヤの実装
重みから特定の行を抜き出すには，単にW[2]やW[5]と書けばよい．

In [3]:
W = np.arange(21).reshape(7, 3)
print(W)
print()
print(W[2])
print(W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]

[6 7 8]
[15 16 17]


複数の行をまとめて抽出する  
ミニバッチ処理を想定

In [4]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

Embeddingレイヤを実装  
順伝播は重みWの特定の行を何の手も加えず抜き出すだけ  
逆伝播は前の層から伝わってきた勾配を，自身の重みの勾配dWのidxで指定されたすべての行に設定する．  
この逆伝播は，$y=xW$という順伝播の式のとき，以下の式で表せる．  
$$ \frac{\partial L}{\partial W} = x^{\top} \frac{\partial L}{\partial y} = x^{\top} dout$$
$x$で1になっている選ばれた行が$dout$で，それ以外は0である行列が勾配$\frac{\partial L}{\partial W}$となる．

In [5]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx # ミニバッチで選ばれた行のインデックス
        out = W[idx]
        return out
    
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         dW[self.idx] = dout # 悪い例 idxは重複を持つ可能性があり，そのときインデックスが小さい行は大きい行で上書きされてしまう．
#         return None

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        # 重複するインデックスのところは，上書きではなく加算を行う．
            # 重みの更新は，Winの各行について行われる．
            # インデックスが例えば2つ重複しているところは，2回学習が行われていることになる
            # すると，2回分の重みの更新を一度に行えばよいため，(Win - dW1) - dW2 = Win - (dW1 + dW2) のように計算する．
            # 従って，インデックスが重複しているところは上書きではなく，加算を行えばよい．
            
        for i, word_id in enumerate(self.idx):
            # dW[word_id] += dout[i]
            np.add.at(dW, self.idx, dout) # 高速
        
        return None

以上でEmbeddingレイヤの実装は終わり．  
CBOWモデルの実装では，MatMulレイヤをEmbeddingレイヤに切り替えればよい．  
これによってメモリの使用量を減らし，さらに無駄な計算を省くことができるようになった．  

# word2vecの改良②
中間層以降の行列の積とSoftmaxレイヤの計算では，Negative Sampling(負例サンプリング)を行う．  
これによって，語彙数がどれだけ多くなっても計算量を一定に抑えられる．  

## 中間層以降の計算の問題点
例えば，出力層を100万次元にすると，以下の点でまだ問題が残っている
- 中間層のニューロンとWoutの積(100×100万の行列計算)
- Softmaxレイヤの計算(100万次元の平均計算)

特に，Softmaxレイヤの計算は以下の式から，その重さがわかる．分母ではexp計算を100万回行なっている．
$$ y_k = \frac{exp(s_k)}{\sum_{i=1}^{1000000}exp(s_i)} $$

## 多値分類から二値分類へ
この手法の鍵となるアイデアは，多値分類を二値分類で近似することにある．  
先ほどの例では，100万単語から正しい単語を1つ選ぶ問題として考えてきたが，これをYes/Noで答えられる問題で近似できないだろうか？  
例えば，コンテキストがYouとGoodbyeのとき，  
「ターゲットとなる単語は何ですか？」を「ターゲットとなる単語はsayですか？」という質問に変えればいい  
そうした場合，出力層にはニューロンをひとつだけ用意すれば事足りる．  
その出力層のニューロンは，「say」であるスコアを出力しさえすればいい．  

中間層と出力側の重み行列の積は，隠れ層の出力「say」に対応するWoutの列のみを内積計算した，スカラー値のみを出力すればよい．  
そのスコアを確率に変換するため，シグモイド関数を適用する．

## シグモイド関数と交差エントロピー誤差
スコアを確率に変換する．
これまでは多値分類問題だったため，Softmax関数を使っていたが，二値分類問題ではシグモイド関数を使う．  
シグモイド関数は今，以下の式で表される．  
$$ y_0 = \frac{1}{1 + exp(-x)},  y_1 = 1 - y_0$$
グラフの形状はS字カーブをしており，yは0~1の値をとる．xが0に近いほどyは0.5に近くなる．  
このとき交差エントロピー誤差は次のようになる．
$$ L = -\sum_{k} t_klogy_k = -(tlog{y} + (1-t)log(1-y)) $$
$t=0$(正解)のときは正解の確率を使って$-logy$が出力され，逆に$t=0$(不正解)のときは不正解の確率を使って$-log(1-y)$が出力される．  
  
  
SigmoidレイヤとCrossEntropyErrorレイヤを合わせて出力層とする．  
このとき，Sigmoidレイヤの逆伝播は，$y-t$になる．  
正解ラベルが1(不正解)のとき，yは1に近いほど更新量が少なくなり，逆に正解ラベルが0(正解)のとき，yは0に近いほど更新量が少なくなる．  

同様にして，ソフトマックス関数と交差エントロピー誤差の組み合わせ，恒等関数と2乗和誤差の組み合わせにも，逆伝播時には$y-t$の値が伝播する．  

## 多値分類から二値分類へ(実装編)
中間層の出力をhとすると，SigmoidWithLossに入力されるのはWoutからsayの部分のみを抜き出すEmbeddingレイヤの出力とhの積になる．
この層をEmbedding Dotレイヤとして実装する．  

In [6]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    # 抜き出し -> 中間層との積．抜き出すことで0の部分を計算しなくていいため高速
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1) # ミニバッチで選ばれた単語の重みと中間層の内積計算
        # out = target_W @ h と同様だが，1行のみなのでこのような形にしている．
        
        self.cache = (h, target_W)
        return out
    
    # 中間層との積 -> 抜き出し．y=xWの逆をやる．
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1) # doutを列ベクトルにする
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W) # 勾配を設定
        dh = dout * target_W 
        return dh

実装の理由は，具体的な値を見るのが早い．

In [7]:
W = np.array(range(21)).reshape(7, 3)
idx = np.array([0, 3, 1])
h = np.array(range(9)).reshape(3, 3)

embed = Embedding(W)
target_W = embed.forward(idx)
out = np.sum(target_W * h, axis=1)

print("W\n", W, '\n')
print("idx\n", idx, '\n')
print("target_W\n", target_W, '\n')
print("h\n", h, '\n')
print("target_W * h\n", target_W * h, '\n')
print("out", out, '\n')

W
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]] 

idx
 [0 3 1] 

target_W
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]] 

h
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

target_W * h
 [[ 0  1  4]
 [27 40 55]
 [18 28 40]] 

out [  5 122  86] 



In [8]:
dout = np.array([1, 1, 1])
print("dout\n", dout, "\n")
dout = dout.reshape(dout.shape[0], 1)
print("dout\n", dout, "\n")

dtarget_W = dout * h
print("dtarget_W\n", dtarget_W, "\n")

embed.backward(dtarget_W) # 勾配を設定
print("embed grads\n", embed.grads, "\n")

dh = dout * target_W 
print("dh\n", dh, "\n")

dout
 [1 1 1] 

dout
 [[1]
 [1]
 [1]] 

dtarget_W
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

embed grads
 [array([[ 0,  3,  6],
       [18, 21, 24],
       [ 0,  0,  0],
       [ 9, 12, 15],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0]])] 

dh
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]] 



## Negative Sampling
今はまだ正例についてだけ学習を行ったにすぎない．  
上の実装は，sayの確率が1に近づくように重みを学習させることはできるが，sayでない単語を0に近づけるように学習させることができない． 
かといって，全てのsayでない単語について学習していては何も変わらない．  
そこで，近似解として少ない数の負例をピックアップして損失を求める．  
そして，正例と負例における損失を全て足し合わせて，その結果を最終的な損失とする．  
以上がNegative Samplingである．  

## Negative Samplingのサンプリング手法
負例は，コーパス中でよく使われる単語を抽出されやすくし，あまり使われない単語は抽出されにくくする．  
コーパスから各単語の出現した回数を求め，これを確率分布で表し，そこから単語を抽出する．  
レアな単語がコンテキストになることは現実的な問題においてもあまりなく，抽出する重要性は低い．  

これには，以下のnumpyによるランダムサンプリングを行う

In [9]:
# 0~9からサンプリング
print(np.random.choice(10))
print(np.random.choice(10))

# words から一つランダムサンプリング
words = ['you', 'say', 'goodbye', 'I', 'Hello', '.']
print(np.random.choice(words))

# wordsから重複をゆるして5つサンプリング
print(np.random.choice(words, size=5))

# wordsから重複をゆるさないで5つサンプリング
print(np.random.choice(words,size=5,replace=False))

#　確率分布pに従ってサンプリング
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

6
3
.
['Hello' 'say' 'say' 'you' 'goodbye']
['Hello' 'you' '.' 'goodbye' 'say']
I


Negative Samplingでは，単語の確率分布に対して0.75乗することが提案されている．  
これにより，出現確率の低い単語の出現確率を相対的に少し上げ，見捨てないようにする．  
ただし，0.75乗した後も確率の和が1になるよう，分母の各要素にも0.75乗する必要がある． 
$$ P'(w_i) = \frac{P(w_i)^{0.75}}{\sum_j^n P(w_j)^{0.75}} $$
なお，0.75に理論的な意味はなく，ほかの値でもいい．　　

In [10]:
print(p)
new_p = np.power(p, 0.75)
new_p = new_p / np.sum(new_p)
print(new_p)
print("new_p sum: ", np.sum(new_p))

[0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
[0.40703871 0.12173297 0.07238286 0.20472963 0.07238286 0.12173297]
new_p sum:  1.0


ここではnegative samplingを入手するためのクラスとしてch04/negative_sampling_layer.pyにあるUnigramSamplerを利用する．  
UnigramSamplerでは，ターゲットとなる単語は確率0としている．  
Unigramという名前がついているのは，1つの単語がターゲットになっているから.  

In [11]:
from ch04.negative_sampling_layer import UnigramSampler

corpus = np.array([0,1,2,3,4,1,2,3]) # 単語のインデックスによるコーパス
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[4 3]
 [1 2]
 [3 2]]


## Negative Samplingの実装
SoftmaxWithLossと同じで，最後の層を担当させる．

In [12]:
from common.layers import SigmoidWithLoss

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] # 0: target, 1~: negative samples
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] # 0: target, 1~: negative samples
        
        # paramsとgradsをそれぞれ同じ要素番号のところで対応するリストにまとめる．
        # リストに対するリストの加算はappendを意味する
        # paramsとgradsをメンバ変数としてまとめて持っておくことで，SGDやAdaGradといったOptimizerに簡単に重みを渡すことができる
        # Optimizerでは，gradsを用いてparamsを更新する．バッチをランダムに選ぶSGDやgradsに反動を加えて局所最適解に陥らないようにするmomentum等
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 正例のフォワード
        correct_label = np.ones(batch_size, dtype=np.int32) # batchの各要素に対して正例ラベルを生成
        score = self.embed_dot_layers[0].forward(h, target) # 埋め込み層(hからtargetの行を抜き出す)の処理
        loss = self.loss_layers[0].forward(score, correct_label) # Sigmoid関数と交差エントロピー誤差によるLossを求める
        
        # 負例のフォワード
        negative_label = np.zeros(batch_size, dtype=np.int32) # batchの各要素に対して負例ラベルを生成
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i] # i列を持ってくる書き方, 複数のtarget(上の例でいうと[1,3,0])に対する各sampleを処理する
            score = self.embed_dot_layers[1 + i].forward(h, negative_target) # 埋め込み層(hからtargetの行を抜き出す)の処理
            loss += self.loss_layers[1 + i].forward(score, negative_label) # Sigmoid関数と交差エントロピー誤差によるLossを求める
        
        return loss
    
    # dout: 前の層から伝わってきたLoss
    def backward(self, dout=1):
        dh = 0
        # 順伝搬のときはembed -> lossだったが，ここでは逆伝搬なので逆にして loss -> embed の順で伝搬する
        # 各サンプルに対して逆伝搬を行う．target(+) -> negative(-) -> negative(-) -> ... これらのsampleはそれぞれbatchなので，それぞれリスト．
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        
        return dh # 前の層に伝えるLoss, 全てのニューロンに伝わる

# 改良版word2vecの学習
PTBデータセットを使って実用的な単語の分散表現を獲得

## CBOWモデルの実装

In [13]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 重みからレイヤを生成
        # CBOWは複数の単語からターゲットを推測するモデルなので，EmbeddingレイヤをContextの単語数だけ並列に並べる
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # すべての重みと勾配を配列にまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in
    
    # CBOWの順伝搬では，学習データとしてコンテキスト，推論対象である正解ラベルとしてターゲットが与えられる．
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target) # ターゲットはロスを求める層に直接渡してやる
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

## CBOWモデルの学習コード
学習には半日程度かかることもある．GPUを使えば早いけど．  
学習したらpickleを使って重み，すなわち単語の分散表現を保存しておく．  
学習済みパラメータがあるので．それを使ってもいい．

In [18]:
from common import config
import pickle
# config.GPU = True # GPUを使うときはTrueにする

from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# ハイパーパラメータの設定
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print(vocab_size)

# コーパスから窓の大きさのコンテキストと，対応するターゲットを取得
contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target) # np array -> cupy array
    
# モデルなどの生成
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

"""
# 学習開始 contextsを訓練データ，targetを正解ラベルとして学習を行う
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 後で利用できるように，必要なデータを保存
word_vecs = model.word_vecs

if config.GPU:
    word_vecs = to_cpu(word_vecs)

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)
"""
print("めっちゃ時間かかるので注意")

10000
めっちゃ時間かかるので注意


## CBOWモデルの評価
2章で実装したmost_similar()メソッドでターゲットとなる単語に対し最も近い距離，すなわち近い意味を持つ単語TOP5を表示してみる

In [15]:
from common.util import most_similar
import pickle

pkl_file = 'ch04/cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


結果を見ると，似たようなポジションの単語と近い意味を持っていることがわかり，なかなか良いベクトル表現になっていることがわかる．  
また，ベクトルの分散表現は類推問題(アナロジー問題)を解けることも分かっている．  
例えば，king - man + woman = queen

In [21]:
from common.util import analogy

# aがbならcは何？
analogy('king', 'man', 'queen', word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs)


[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875


最初の三つはうまくいっているが，badはダメだった．しかし，比較級が並んでいることから，文法的な意味は捉えられている模様  
他にも，goodとbestの間にはbetterが存在するといった関係性など，word2vecの単語の分散表現には興味深い結果がいくつも見つかっている．  
しかしPTBデータセットは小規模なので期待する結果が得られないことが多い．さらに大きなコーパスを対象として学習すれば，より精度が高く，よりロバストな単語の分散表現を得られ，類推問題の正解率も向上することが期待できる．

# word2vecに関する残りのテーマ

## word2vecを使ったアプリケーションの例

### 転移学習
ある分野で学んだ知識を他の分野にも適用すること．  
自然言語のタスクでは，単語の分散表現をゼロから学習することはなく，先に大きなコーパスで行った学習結果を個別のタスクで利用する．
- テキスト分類
- 文書クラスタリング
- 品詞タグ付け
- 感情分析

### 文章を固定長のベクトルに変換する
単語の固定長ベクトルへの変換を見てきたが，文章はどうなるだろうか  
- bag of words: 文章内の単語の固定長のベクトルの長さ
- リカレントニューラルネットワーク(5章)

自然言語をベクトルに変換できれば，一般的な機械学習の手法が適用できるため，重要である．  
しかし手元に大量のデータがあるならゼロから単語の分散表現を学習することもアリかもしれない．  

### アプリケーションの例
1. メールデータを用意
2. メールデータに感情のラベル付け
3. 学習済みのword2vecを使って(転移学習)メールデータをベクトルへ変換
4. 感情分析を行うSVMやNNといった分類システムで分類  

このように転移学習を行うことで，早く高い精度のタスク処理が可能になる

## 単語ベクトルの評価方法
単語の分散表現を使ったタスク処理では，システム全体の出す結果をもとに評価を行う必要があるが，手間がかかる．  
そこで，「類似性」や「類推問題」を使って単語の分散表現は評価される．  

- 類似性の評価
    - 人の手で2つの単語の類似度を10段階などでスコア化する
- 類推問題の評価
    - こちらも人の手で king:queen = man:? のような類推問題を解かせたデータを使う
    - semanticsとsyntaxで分けて評価されたりする

論文[27]では以下のことが分かっている．
- CBOWかskip-gramか，モデルによって精度が異なるので，コーパスに応じて最適なモデルを選ぶことが必要
- コーパスが大きいほどいい結果になる
- 単語ベクトルの次元数は大きすぎても小さすぎてもよくない，適切な大きさが必要

また，類推問題の評価が高いからと言って，その分散表現を使った次の学習でも必ず良い結果になるとは限らないので注意する